In [ ]:

import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/Dataset.csv

--2023-07-17 14:07:44--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/Dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 880938 (860K) [text/plain]
Saving to: ‘Dataset.csv’

Dataset.csv         100%[===================>] 860.29K  --.-KB/s    in 0.05s   

2023-07-17 14:07:44 (17.9 MB/s) - ‘Dataset.csv’ saved [880938/880938]



In [ ]:
#df = pd.read_csv('eng-tam_3_train.csv',sep='\t',names=['category','text']) # separator was not given properly
# !wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/Dataset.csv
# from google.colab import drive
# drive.mount('/content/gdrive')
df = pd.read_csv('/content/Dataset.csv')
#df = pd.read_csv('eng-tam_3_train.csv')
text=df[['text']]
labels=df[['category']]

In [ ]:
text

,text
0,Viswasam trailer paathutu Inka vanthavanga lik...
1,Thalaiva.. full Support from kerala Mohanlal Fans
2,Paruthiveeran Karthi Back Lokesh Kangaraj Scr...
3,Hey Makkalay yaaru Darbar Motion Poster announ...
4,Padaiyatchi nu sonna mattum ungalukku jaathi v...
...,...
12590,Aiiiiiiiyo vera leval Hollywood ranguku ierukk...
12591,Rip Sr Prabhu punda mavan... Ne theru Kodi th...
12592,Trailer pudichuruku nu solluravunga hit like..
12593,Oruthar mela viswasam kaatta yen inoruthara as...


In [ ]:
labels

,category
0,unknown_state
1,unknown_state
2,unknown_state
3,unknown_state
4,unknown_state
...,...
12590,Mixed_feelings
12591,Mixed_feelings
12592,Mixed_feelings
12593,Mixed_feelings


In [ ]:
import re
import nltk
nltk.download('punkt', 'stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to stopwords...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def take_data_to_shower(tweet):
    tweet=str(tweet)

    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [ ]:
def remove_emoticon(text):
        text=str(text)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [ ]:
 def remove_punc(text):
    text=str(text)
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [ ]:
def tokenize(tweet):
   tweet=str(tweet)

   lower_tweet = tweet.lower()
   return word_tokenize(lower_tweet)

In [ ]:
def remove_stop_words(tokens):
   clean_tokens = []
   newStopWords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's', 'co','oru', 'enru', 'matrum', 'indha', 'idhu', 'enru', 'kondu', 'enpathu', 'pala', 'aagum', 'allathu', 'avar', 'naan', 'ullu', 'andha', 'ivar', 'ena', 'muthal', 'enna', 'irundhu', 'sila', 'en', 'pontu', 'venṭum', 'vandhu', 'ithan', 'athu', 'avan', 'thaan', 'palarum', 'ennum', 'melum', 'pinar', 'kondu', 'irukkum', 'thanathu', 'ullathu', 'pothu', 'enrum', 'athan', 'than', 'piragu', 'avargal', 'varai', 'aval', 'nee', 'aagiyathu', 'ullana', 'vandha', 'irundh', 'migavum', 'inggu', 'meethu', 'or', 'ivai', 'indha', 'ithaṉaḷ', 'patri', 'varum', 'veṟu', 'iru', 'ithil', 'pol', 'ippothu', 'avarathu', 'maṭṭum', 'indha', 'enuṁ', 'mel', 'pin', 'cernta', 'ākiyōr', 'enakku', 'innum', 'andha', 'anṟu', 'oṟe', 'miga', 'angu', 'palvēru', 'viṭṭu', 'perum', 'athai', 'paṟṟiya', 'uṉ', 'athik', 'andhak', 'pēr', 'indhaṉāḷ', 'avai', 'athe', 'ēn', 'muṟai', 'yār', 'eṉappaṭum', 'ellām', 'maṭṭumē', 'ingkē', 'angkē', 'iṭam', 'iṭattil', 'athil', 'nām', 'atharku', 'eṉavē', 'piṟa', 'siṟu', 'maṟṟa', 'viṭa', 'endha', 'eṉavum', 'eṉappuṭum', 'eṉiṉum', 'aṭutta', 'ithanai', 'ithai', 'koḷḷa', 'indhath', 'ithaṉāl', 'athāl', 'thavira', 'pol', 'varail', 'caṟṟu', 'eṉak']
   stopWords = stopwords.words('english')
   stopWords.append(newStopWords)
  #  print(stopWords)

   for token in tokens:
       if token not in stopWords:
           if token.replace(' ', '') != '':
               if len(token) > 1:
                   clean_tokens.append(token)
   return clean_tokens

In [ ]:
def stem_and_lem(tokens):
   clean_tokens = []
   for token in tokens:
       token = wordnet_lemmatizer.lemmatize(token)
       token = lancaster_stemmer.stem(token)
       if len(token) > 1:
           clean_tokens.append(token)
   return clean_tokens


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
##EMBEDDING##
clean_texts = copy.deepcopy(text)
tqdm.pandas(desc="Cleaning Data Phase I...")
clean_texts['text'] = text['text'].progress_apply(take_data_to_shower)

tqdm.pandas(desc="removing emoji...")
clean_texts['text'] = text['text'].progress_apply(remove_emoticon)


tqdm.pandas(desc="removing punctuation...")
clean_texts['text'] = text['text'].progress_apply(remove_punc)



tqdm.pandas(desc="Tokenizing Data...")
clean_texts['tokens'] = clean_texts['text'].progress_apply(tokenize)

tqdm.pandas(desc="Cleaning Data Phase II...")
clean_texts['tokens'] = clean_texts['tokens'].progress_apply(remove_stop_words)

tqdm.pandas(desc="Stemming And Lemmatizing")
clean_texts['tokens'] = clean_texts['tokens'].progress_apply(stem_and_lem)



text_vector = clean_texts['tokens'].tolist()

Stemming And Lemmatizing: 100%|██████████| 12595/12595 [00:03<00:00, 3608.27it/s]


In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer

def tfid(text_vector):
    vectorizer = TfidfVectorizer()
    untokenized_data =[' '.join(tweet) for tweet in tqdm(text_vector, "Vectorizing...")]
    vectorizer = vectorizer.fit(untokenized_data)
    vectors = vectorizer.transform(untokenized_data).toarray()
    return vectors

def get_vectors(vectors, labels, keyword):
    if len(vectors) != len(labels):
        print("Unmatching sizes!")
        return
    result = list()
    for vector, label in zip(vectors, labels):
        if label == keyword:
            result.append(vector)
    return result

In [ ]:
vectors_a = tfid(text_vector) # Numerical Vectors A
labels_a = labels['category'].values.tolist() # Subtask A Labels

Vectorizing...: 100%|██████████| 12595/12595 [00:00<00:00, 593711.47it/s]


In [ ]:
vectors_a

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
##CLASSIFING##

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
def classify(vectors, labels, type="DT"):


  # Random Splitting With Ratio 3 : 1
  train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, random_state=5, test_size=0.2)

  # Initialize Model
  classifier = None
  if(type=="MNB"):
    classifier = MultinomialNB(alpha=0.7)
    classifier.fit(train_vectors, train_labels)
  elif(type=="KNN"):
    classifier = KNeighborsClassifier(n_jobs=4)
    params = {'n_neighbors': [3], 'weights':['uniform', 'distance']}
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="SVM"):
    classifier = SVC()
    classifier = GridSearchCV(classifier, {'C':[0.001, 0.01, 0.1, 1, 10]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="DT"):
    classifier = DecisionTreeClassifier()
    #params = {'criterion':['gini','entropy']}
    #classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    #classifier = classifier.best_estimator_
  elif(type=="RF"):
    classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
    params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="LR"):
    classifier = LogisticRegression(multi_class='auto', solver='newton-cg')
    classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="XG"):
    classifier=XGBClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="AD"):
    classifier=AdaBoostClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="VE"):
    estimators = []
    model1 = LogisticRegression()
    estimators.append(('logistic', model1))
    model2 = DecisionTreeClassifier()
    estimators.append(('cart', model2))
    classifier = VotingClassifier(estimators)
    classifier.fit(train_vectors, train_labels)

  else:
    print("Wrong Classifier Type!")
    return

  accuracy = accuracy_score(train_labels, classifier.predict(train_vectors))
  print("Training Accuracy:", accuracy)
  test_predictions = classifier.predict(test_vectors)
  accuracy = accuracy_score(test_labels, test_predictions)
  print("Test Accuracy:", accuracy)
  print("Confusion Matrix:", )
  print(confusion_matrix(test_labels, test_predictions))
  print(classification_report([i for i in test_labels],
                        [i for i in test_predictions]))

In [ ]:
print("\nBuilding Model SVM...")
classify(vectors_a, labels_a, "SVM") # {MNB, KNN, SVM, DT, RF, LR,XG,AD,VE}

In [ ]:
print("\nBuilding Model MNB...")
classify(vectors_a, labels_a, "MNB")

In [ ]:
print("\nBuilding Model KNN...")
classify(vectors_a, labels_a, "KNN")



Building Model KNN...


In [ ]:
print("\nBuilding Model DT...")
classify(vectors_a, labels_a, "DT")



Building Model DT...
Training Accuracy: 0.9970226280269948
Test Accuracy: 0.6081778483525209
Confusion Matrix:
[[  34   32  200    6   13]
 [  29   61  214    6   14]
 [ 125  111 1379   28   35]
 [   4    2   38   39    2]
 [   9   10  105    4   19]]
                precision    recall  f1-score   support

Mixed_feelings       0.17      0.12      0.14       285
      Negative       0.28      0.19      0.23       324
      Positive       0.71      0.82      0.76      1678
     not-Tamil       0.47      0.46      0.46        85
 unknown_state       0.23      0.13      0.17       147

      accuracy                           0.61      2519
     macro avg       0.37      0.34      0.35      2519
  weighted avg       0.56      0.61      0.58      2519



In [ ]:
print("\nBuilding Model RF...")
classify(vectors_a, labels_a, "RF")

In [ ]:
print("\nBuilding Model LR...")
classify(vectors_a, labels_a, "LR")


Building Model LR...
Training Accuracy: 0.7379912663755459
Test Accuracy: 0.684795553791187
Confusion Matrix:
[[   0    4  280    1    0]
 [   1   30  288    2    3]
 [   3    7 1660    7    1]
 [   0    1   56   28    0]
 [   2    3  134    1    7]]
                precision    recall  f1-score   support

Mixed_feelings       0.00      0.00      0.00       285
      Negative       0.67      0.09      0.16       324
      Positive       0.69      0.99      0.81      1678
     not-Tamil       0.72      0.33      0.45        85
 unknown_state       0.64      0.05      0.09       147

      accuracy                           0.68      2519
     macro avg       0.54      0.29      0.30      2519
  weighted avg       0.60      0.68      0.58      2519



In [ ]:
print("\nBuilding Model XGB...")
classify(vectors_a, labels_a, "XG")

In [ ]:
print("\nBuilding Model AD...")
classify(vectors_a, labels_a, "AD")

In [ ]:
print("\nBuilding Model VE...")
classify(vectors_a, labels_a, "VE")